# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import pandas as pd

In [ ]:
#| export
def move_columns(
    df: pd.DataFrame,  # Input
    cols_to_move: str,  # Single 
    pos: int  # Target
) -> pd.DataFrame:
    """
    Move one or more columns to a specified position in a DataFrame.
    """

    # Ensure `cols_to_move` is always a list
    if isinstance(cols_to_move, str):
        cols_to_move = [cols_to_move]

    cols = list(df.columns)

    # Remove the columns to be moved from the current order
    for c in cols_to_move:
        cols.remove(c)

    # Insert the removed columns after the specified position
    new_cols = cols[:pos] + cols_to_move + cols[pos:]

    # Return the reordered DataFrame
    return df[new_cols]


In [ ]:
df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]})
df

,A,B,C
0,1,4,7
1,2,5,8
2,3,6,9


In [ ]:
move_columns(df, 'C', 0)

,C,A,B
0,7,1,4
1,8,2,5
2,9,3,6


## Wrapper 

In [ ]:
a = 10

In [ ]:
def my_sum(b):
    return a + b

my_sum(32)

42

In [ ]:
import inspect

In [ ]:
# inspect.getclosurevars(eval("my_sum"))
inspect.getclosurevars(my_sum)

ClosureVars(nonlocals={}, globals={'a': 10}, builtins={}, unbound=set())

In [ ]:
def my_callback(result):
    print("Cell just finished running!")

get_ipython().events.register('post_run_cell', my_callback)

Cell just finished running!


In [ ]:
get_ipython().events.unregister('post_run_cell', my_callback)

In [ ]:
import ast

def check_funcs(result):
    tree = ast.parse(result.info.raw_cell)
    func_names = [node.name for node in ast.walk(tree) if isinstance(node, ast.FunctionDef)]
    if func_names:
        print(f"Functions defined: {func_names}")

get_ipython().events.register('post_run_cell', check_funcs)

Functions defined: ['check_funcs']


In [ ]:
get_ipython().events.unregister('post_run_cell', check_funcs)

In [ ]:
def check_funcs(result):
    tree = ast.parse(result.info.raw_cell)
    func_names = [node.name for node in ast.walk(tree) if isinstance(node, ast.FunctionDef)]
    if func_names:
        print(f"Functions defined: {func_names}")

get_ipython().events.register('post_run_cell', check_funcs)

Functions defined: ['check_funcs']


In [ ]:
get_ipython().events.unregister('post_run_cell', check_funcs)

In [ ]:
import ast
import inspect

In [ ]:
def check_global_deps(result):
    tree = ast.parse(result.info.raw_cell)
    func_names = [node.name for node in ast.walk(tree) if isinstance(node, ast.FunctionDef)]
    
    ns = get_ipython().user_ns
    for name in func_names:
        if name in ns:
            func = ns[name]
            cv = inspect.getclosurevars(func)
            if cv.globals:
                print(f"⚠️  '{name}' depends on global variables: {list(cv.globals.keys())}")

In [ ]:
get_ipython().events.register('post_run_cell', check_global_deps)

In [ ]:
def my_sum(b):
    return a + b

my_sum(32)

42

⚠️  'my_sum' depends on global variables: ['a']


In [ ]:
get_ipython().events.unregister('post_run_cell', check_global_deps)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()